<a href="https://colab.research.google.com/github/chuckfinca/arithmephi/blob/main/arithmephi_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==2.1.0
!pip install pyarrow==14.0.2
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers
# wiki dump had an issue which may require datasets==2.8.0 but i am hoping it does not
!pip install einops multiprocess==0.70.13 datasets bitsandbytes accelerate peft flash_attn num2words

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-r945d4vh
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-r945d4vh
  Resolved https://github.com/huggingface/transformers to commit 923733c22bf4d3cc6661c8cd3b730b275e9a938e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8660827 sha256=097f881c82117b96270e5dcc3d455008fc0b886837b02c775d6316d3d17829d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-1izngx24/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.7 MB/s eta 0:00:00
     ━━━━

In [2]:
import flash_attn, torch, locale, pandas, time, random, string, num2words, datasets
# from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

In [3]:
# from huggingface_hub import notebook_login
# notebook_login()
from huggingface_hub import login
login()

In [4]:
model_name = "microsoft/phi-2"#"chuckfinca/arithmephi"

# Configuration to load model in 4-bit quantized
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_compute_dtype='float16',
                                #bnb_4bit_compute_dtype=torch.bfloat16,
                                bnb_4bit_use_double_quant=True)

#Loading Microsoft's Phi-2 model with compatible settings
#Remove the attn_implementation = "flash_attention_2" below to run on T4 GPU
# V100 isn't supported by either flash_attention or flash_attention_2
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto',
                                             quantization_config=bnb_config,
                                            #  attn_implementation="flash_attention_2",
                                             trust_remote_code=True)

# Setting up the tokenizer for Phi-2
tokenizer_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name,
                                          add_eos_token=True,
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# def response(query):
#   inputs = tokenizer(query, return_tensors="pt", return_attention_mask=False)
#   outputs = model.generate(**inputs, max_length=len(inputs[0]) + 8, pad_token_id=tokenizer.eos_token_id)
#   text = tokenizer.batch_decode(outputs)[0]
#   return text

In [6]:
# def counting_test():
#   add_typos = True
#   prompt, answer = generate_counting_prompt_and_answer(-1 * max_int, max_int, add_typos)

#   response_string = response(prompt).replace(prompt, "")
#   return prompt, answer, response_string

In [7]:
percent_typos = 0.01

In [8]:
def generate_counting_prompt_and_answer(min, max, add_typos):
    number = random.randint(min, max)
    prompt = ""
    for i in range(3):
        if prompt != "":
            prompt += " "

        if random.getrandbits(1):
            prompt += num2words.num2words(float(number))
            if add_typos:
              prompt = insert_typos(prompt, percent_typos)
        else:
            prompt += f"{number}"
        prompt += ","
        number += 1
    return prompt, f"{number}"

In [9]:
# def extract_number(response):
#   first_response = response.split(", ")[0]
#   array = first_response.split(" ")
#   if array[0] == '':
#     array = array[1:]
#   try:
#     return int(array[0])
#   except:
#     return response.split("\n")[0].split(",")[0].split("<|endoftext|>")[0]

In [10]:
def insert_typos(phrase, probability_of_typo):
    # TODO: this can be made better using common misspellings like those found https://www.dcs.bbk.ac.uk/~ROGER/corpora.html
    new_phrase = []
    words = phrase.split(' ')
    for word in words:
        outcome = random.random()
        if outcome <= probability_of_typo:
            typo_type = random.choice(['insert', 'omit', 'replace', 'replace'])
            ix = random.choice(range(len(word)))
            if typo_type == 'replace':
                new_word = ''.join([word[w] if w != ix else random.choice(string.ascii_letters) for w in range(len(word))])
            elif typo_type == 'insert':
                new_word = ''.join([word[w] if w != ix else word[w] + random.choice(string.ascii_letters) for w in range(len(word))]) + (random.choice(string.ascii_letters) if ix == len(word) - 1 else '')
            elif typo_type == 'omit' and len(word) > 1:
                new_word = ''.join([word[w] for w in range(len(word)) if w != ix])
            else:
                new_word = word
            new_phrase.append(new_word)
        else:
            new_phrase.append(word)

    return ' '.join([w for w in new_phrase])

In [11]:
# start_time = time.time()

# # Initialize lists
# prompts = []
# answers = []
# extracted_responses = []

# score = 0
# number_of_questions = 15
# for i in range(number_of_questions):
#   prompt, answer, extracted_response = counting_test()
#   print(f"iteration: {i}")

#   print(f"prompt: {prompt}")
#   print(f"answer: {answer}")

#   number_response = extract_number(extracted_response)
#   print(f"extracted_response: {number_response}")
#   try:
#     if int(number_response) == int(answer):
#       score += 1
#     else:
#       print("❌")
#   except:
#       print("❌")


#   prompt, answer, extracted_response = counting_test()
#   prompts.append(prompt)
#   answers.append(answer)
#   extracted_responses.append(extract_number(extracted_response))

# print(f"Score: {score}/{number_of_questions}")

# # Create DataFrame
# df = pandas.DataFrame({
#     'Prompt': prompts,
#     'Answer': answers,
#     'Extracted Response': extracted_responses
# })

# # Export to CSV
# df.to_csv('/content/drive/MyDrive/Machine Learning/arithmephi/counting_ft1_2M.csv', index=False)

In [12]:
from peft import prepare_model_for_kbit_training

#gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# Freeze base model layers and cast layernorm in fp32
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
print(model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layern

In [13]:
#Accelerate training models on larger batch sizes, we can use a fully sharded data parallel model.
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [14]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [15]:
from peft import prepare_model_for_kbit_training

print_trainable_parameters(model)

#gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# Freeze base model layers and cast layernorm in fp32
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
print(model)

trainable params: 0 || all params: 1521392640 || trainable%: 0.00
PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropo

In [16]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
    'q_proj',
    'k_proj',
    'v_proj',
    'dense',
    'fc1',
    'fc2',
    ], #print(model) will show the modules to use
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

lora_model = get_peft_model(model, config)

lora_model = accelerator.prepare_model(lora_model)

In [17]:
#Function that creates a prompt from instruction, context, category and response and tokenizes it
def collate_and_tokenize(examples):

    question = examples["question"][0].replace('"', r'\"')
    answer = examples["answer"][0].replace('"', r'\"')
    prompt = question + " " + answer

    # print(f"prompt: {prompt}")

    #Tokenize the prompt
    encoded = tokenizer(
        prompt,
        return_tensors="np",
        padding="max_length",
        truncation=True,
        ## Very critical to keep max_length at 1024 on T4
        ## Anything more will lead to OOM on T4
        max_length=1024,
    )

    encoded["labels"] = encoded["input_ids"]
    return encoded

In [18]:
def create_counting_data(count, min_int, max_int):
  # Create an empty DataFrame with the specified columns
  df = pandas.DataFrame(columns=['question', 'answer'])

  for i in range(count):
    prompt, answer = generate_counting_prompt_and_answer(min_int, max_int, True)
    example = [prompt, answer]
    df.loc[i] = example

  return df

In [19]:
# !pip install mathematics_dataset

In [20]:
# import mathematics_dataset

In [21]:
# mathematics_dataset.g

In [22]:
# from https://huggingface.co/datasets/olm/wikipedia
# !pip install mwparserfromhell==0.6.4 #multiprocess==0.70.13

In [23]:
# from datasets import load_dataset

# load_dataset("olm/wikipedia", language="en", date="20240220")

In [24]:
# from google.colab import files

# files.upload()

In [25]:
!pip install kaggle

In [26]:
# setup the kaggle directory if necessary
!mkdir ~/.kaggle

# copy the json file to the directory
!cp 'drive/MyDrive/Machine Learning/arithmephi/kaggle.json' ~/.kaggle/

In [27]:
# change the file permissions
! chmod 600 ~/.kaggle/kaggle.json

In [28]:
!kaggle datasets download -d mikeortman/wikipedia-sentences

100% 313M/314M [00:15<00:00, 24.0MB/s]
100% 314M/314M [00:15<00:00, 21.7MB/s]


In [29]:
!unzip wikipedia-sentences.zip

Archive:  wikipedia-sentences.zip
  inflating: wikisent2.txt           


In [30]:
#!head -15 wikisent2.txt

In [31]:
#!wc -l wikisent2.txt

In [32]:
def create_wiki_sentences_data(count):
  # Create an empty DataFrame with the specified columns
  df = pandas.DataFrame(columns=['question', 'answer'])

  file = open('wikisent2.txt')
  content = file.readlines()
  for i in range(count):
    prompt, answer = generate_wiki_sentence_prompt_and_answer(content, True)
    example = [prompt, answer]
    df.loc[i] = example

  return df

In [33]:
def generate_wiki_sentence_prompt_and_answer(content, add_typos):

    # # read 10th line from the file
    # print("tenth line")
    # print(content[9])
    # print(content[0])
    # print(content[7871824])


    # # print first 3 lines of file
    # print("first three lines")
    # print(content[0:3])

    # print("---")

    # dataset is 7871825 items
    number = random.randint(0, 7871824)

    line = content[number]
    array = line.split(" ")

    if len(array) > 15 and random.randint(0,1):
      half = len(array) // 2
      offset = half + random.randint(0, half - 1)
      array = array[:offset]

    last_index = len(array) - 1
    answer = array.pop()
    answer = answer.replace("\n", "").replace(".", "")
    prompt = " ".join(array)
    prompt = insert_typos(prompt, percent_typos)
    return prompt, answer

In [34]:
min_int = 0
max_int = 2500
train_counting_dataframe = create_counting_data(3000, min_int, max_int)
test_counting_dataframe = create_counting_data(600, min_int, max_int)

train_wiki_dataframe = create_wiki_sentences_data(2000)
test_wiki_dataframe = create_wiki_sentences_data(400)

train_frames = [train_counting_dataframe, train_wiki_dataframe]
unshuffled_train_data = pandas.concat(train_frames)
train_data = unshuffled_train_data.sample(frac=1)

test_frames = [test_counting_dataframe, test_wiki_dataframe]
unshuffled_test_data = pandas.concat(test_frames)
test_data = unshuffled_test_data.sample(frac=1)

train_dataset = datasets.Dataset.from_pandas(train_data)
test_dataset = datasets.Dataset.from_pandas(test_data)

# print(train_dataset)
# print(train_dataset.shape)
# display(train_dataset)
# display(train_data)

In [35]:
#We will just keep the input_ids and labels that we add in function above.
columns_to_remove = ["question","answer"]

#tokenize the training and test datasets
tokenized_dataset_train = train_dataset.map(collate_and_tokenize,
                                            batched=True,
                                            batch_size=1,
                                            remove_columns=columns_to_remove)
tokenized_dataset_test = test_dataset.map(collate_and_tokenize,
                                          batched=True,
                                          batch_size=1,
                                          remove_columns=columns_to_remove)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [36]:
commit_message = f"Training run @ {time.time()}"

In [ ]:
import time
from transformers import TrainingArguments, Trainer

feedback_steps = 50
batch_size = 3

training_args = TrainingArguments(
    output_dir='./results',  # Output directory for checkpoints and predictions
    overwrite_output_dir=True, # Overwrite the content of the output directory
    per_device_train_batch_size=batch_size,  # Batch size for training
    per_device_eval_batch_size=batch_size,  # Batch size for evaluation
    gradient_accumulation_steps=5, # number of steps before optimizing
    gradient_checkpointing=True,   # Enable gradient checkpointing
    gradient_checkpointing_kwargs={"use_reentrant": False},
    # From ChatGPT: In the context of machine learning, especially when training large models like transformers, controlling the learning rate is crucial for effective training. A common approach is to start with a smaller learning rate and gradually increase it to a maximum value. This process is known as warmup. The purpose is to prevent the model from converging too quickly to a suboptimal solution at the start of training when weights are initialized randomly.
    warmup_steps=50,  # Number of warmup steps
    #max_steps=1000,  # Total number of training steps
    num_train_epochs=2,  # Number of training epochs
    learning_rate=5e-5,  # Learning rate
    weight_decay=0.01,  # Weight decay
    optim="paged_adamw_8bit", #Keep the optimizer state and quantize it
    fp16=True, #Use mixed precision training
    #For logging and saving
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=feedback_steps,
    save_strategy="steps",
    save_steps=feedback_steps,
    save_total_limit=2,  # Limit the total number of checkpoints
    evaluation_strategy="steps",
    eval_steps=feedback_steps,
    load_best_model_at_end=True, # Load the best model at the end of training
)

trainer = Trainer(
    model=lora_model,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    args=training_args,
)

#Disable cache to prevent warning, renable for inference
model.config.use_cache = False

start_time = time.time()  # Record the start time
trainer.train()  # Start training
end_time = time.time()  # Record the end time

training_time = end_time - start_time  # Calculate total training time

print(f"Training completed in {training_time} seconds.")

#Save model to hub to ensure we save our work.
lora_model.push_to_hub("arithmephi",
                  use_auth_token=True,
                  commit_message="Testing new wiki & counting training setup", #commit_message,
                  private=True)

Step,Training Loss,Validation Loss
50,3.050700,0.067873
100,0.055900,0.045821
